In [2]:
%load_ext autoreload
%autoreload 1
%aimport prediction_models

import numpy as np 
import pandas as pd 
import json
from pandas.io.json import json_normalize
import nltk
import copy
import random
from collections import Counter
import operator
import math

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def get_utterances(item):
    
    utterances = []
    
    for act in item['acts']:
        
        if (act['text'] not in ['Submit-Deal', 'Accept-Deal', 'Reject-Deal', 'Walk-Away', 'Submit-Post-Survey']):
            utterances.append(act['text'])
            
    return utterances

def remove_emo_from_utter(txt):
    return txt.replace('🙂', ' ').replace('😮', ' ').replace('☹️', ' ').replace('😡', ' ')

def remove_emoticons(all_utterances):
    
    new_all_utterances = []
    for utter in all_utterances:
        new_all_utterances.append(remove_emo_from_utter(utter))
    
    return new_all_utterances

In [4]:
in_f = "/home/ICT2000/chawla/Documents/internship2020/git_repo/storage/airtable/all_data-1030-shuffled.json" 
with open(in_f) as f:
    all_data = json.load(f)
    
print(len(all_data))
print(all_data[0].keys())

1030
dict_keys(['convo_is_finished', 'world_tag', 'bad_workers', 'acts', 'turns', 'workers', 'fpath', 'qualtrics', 'dialogue_id'])


In [12]:
model_obj = prediction_models.EmotionTwitterDataModel()

In [17]:
num_utts = 100000
rem_emoticons = True#True

all_utterances = []

for item in all_data:
    all_utterances += get_utterances(item)

if(rem_emoticons):
    all_utterances = remove_emoticons(all_utterances)

random.shuffle(all_utterances)

print(len(all_utterances), len(all_utterances)/len(all_data))

utterance2pred = {}

for i, utt in enumerate(all_utterances[:num_utts]):
    if(not i%500):
        print(i)
    pred = model_obj.predict(utt)
    utterance2pred[utt] = pred
    
print(len(utterance2pred))
    
with open("/home/ICT2000/chawla/Documents/internship2020/git_repo/storage/emotion_predictions.json", 'w') as fp:
    json.dump(utterance2pred, fp)

11919 11.571844660194175
0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
11589


In [20]:
for utt, pred in utterance2pred.items():
    if(pred not in ['joy', 'anger', 'sadness', 'love', 'fear', 'surprise']):
        utterance2pred[utt] = 'fear'

In [21]:
for utt, pred in utterance2pred.items():
    if(pred not in ['joy', 'anger', 'sadness', 'love', 'fear', 'surprise']):
        print(utt, '#', pred)

In [22]:
with open("/home/ICT2000/chawla/Documents/internship2020/git_repo/storage/emotion_predictions.json", 'w') as fp:
    json.dump(utterance2pred, fp)